# Logistic Regression

# **Logistic Regression**

*Colab Environ*

In [ ]:
!apt-get update

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (143 kB/s)
Reading package lists... Done


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

Step **1**: Create the Spark Session Object

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg').getOrCreate()

Step **2**: Read the Dataset

In [ ]:
df = spark.read.csv('Log_Reg_dataset.csv',inferSchema=True,header=True)

Step **3**: Exploratory Data Analysis

In [ ]:
print((df.count(),len(df.columns)))

(20000, 6)


In [ ]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



As we can see, there are two such columns (Country, Platform),
which are categorical in nature and hence need to be converted into
numerical form. Let’s have a look at the dataset using the show function in
Spark.

In [ ]:
df.show(5)

+---------+---+--------------+--------+----------------+------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|    India| 41|             1|   Yahoo|              21|     1|
|   Brazil| 28|             1|   Yahoo|               5|     0|
|   Brazil| 40|             0|  Google|               3|     0|
|Indonesia| 31|             1|    Bing|              15|     1|
| Malaysia| 32|             0|  Google|              15|     1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



We can now use the describe function to go over statistical measures of
the dataset.

In [ ]:
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



We can observe that the average age of visitors is close to 28 years, and
they view around 9 web pages during the website visit.

Let us explore individual columns to understand the data in deeper
details. The **groupBy** function used along with counts returns the
frequency of each of the categories in the data.

In [ ]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



So, the maximum number of visitors are from Indonesia, followed by
India:

In [ ]:
df.groupBy('Platform').count().show()
#The Yahoo search engine users are the highest in numbers.

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [ ]:
df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1|10000|
|     0|10000|
+------+-----+



We have an equal number of users who are converted and nonconverted.

Let’s use the groupBy function along with the mean to know more
about the dataset.

In [ ]:
df.groupBy('Country').mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+



We have the highest conversion rate from Malaysia, followed by India.
The average number of web page visits is highest in Malaysia and lowest in
Brazil.

In [ ]:
df.groupBy('Platform').mean().show()

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+---------------------+------------------+
|   Yahoo|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+-------------------+---------------------+------------------+



We have the highest conversion rate from user visitors use the Google
search engine.

In [ ]:
df.groupBy('Status').mean().show()

+------+--------+-------------------+---------------------+-----------+
|Status|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+--------+-------------------+---------------------+-----------+
|     1| 26.5435|             0.7019|              14.5617|        1.0|
|     0| 30.5356|             0.3039|               4.5449|        0.0|
+------+--------+-------------------+---------------------+-----------+



We can clearly see there is a strong connection between the conversion
status and the number of pages viewed along with repeat visits.

## Step 4: Feature Engineering

This is the part where we convert the categorical variable into numerical
form and create a single vector combining all the input features by using
Spark’s **VectorAssembler**

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

Since we are dealing with two categorical columns, we will have to
convert the country and Platform columns into numerical form

*The first step is to label the column using StringIndexer into
numerical form*

In [ ]:
platform_indexer = StringIndexer(inputCol='Platform',outputCol='Platform_num').fit(df)
df = platform_indexer.transform(df)


In [ ]:
df.show(5)

+---------+---+--------------+--------+----------------+------+------------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_num|
+---------+---+--------------+--------+----------------+------+------------+
|    India| 41|             1|   Yahoo|              21|     1|         0.0|
|   Brazil| 28|             1|   Yahoo|               5|     0|         0.0|
|   Brazil| 40|             0|  Google|               3|     0|         1.0|
|Indonesia| 31|             1|    Bing|              15|     1|         2.0|
| Malaysia| 32|             0|  Google|              15|     1|         1.0|
+---------+---+--------------+--------+----------------+------+------------+
only showing top 5 rows



The next step is to represent each of these values into the form of a
one hot encoded vector. However, this vector is a little different in terms of
representation as it captures the values and position of the values in the vector.

In [ ]:
from pyspark.ml.feature import OneHotEncoder
platform_encoder = OneHotEncoder(inputCol='Platform_num',outputCol='Platform_vector')
df =platform_encoder.transform(df)

In [ ]:
df.show(4)

+---------+---+--------------+--------+----------------+------+------------+---------------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Platform_num|Platform_vector|
+---------+---+--------------+--------+----------------+------+------------+---------------+
|    India| 41|             1|   Yahoo|              21|     1|         0.0|  (2,[0],[1.0])|
|   Brazil| 28|             1|   Yahoo|               5|     0|         0.0|  (2,[0],[1.0])|
|   Brazil| 40|             0|  Google|               3|     0|         1.0|  (2,[1],[1.0])|
|Indonesia| 31|             1|    Bing|              15|     1|         2.0|      (2,[],[])|
+---------+---+--------------+--------+----------------+------+------------+---------------+
only showing top 4 rows



In [ ]:
df.groupBy('platform_Vector').count().orderBy('count',ascending=False).show(5,False)

+---------------+-----+
|platform_Vector|count|
+---------------+-----+
|(2,[0],[1.0])  |9859 |
|(2,[1],[1.0])  |5781 |
|(2,[],[])      |4360 |
+---------------+-----+



The final feature that we would be using for building Logistic
Regression is platform_Vector. Let’s understand what these column
values represent

(2,[0],[1.0]) represents a vector of length 2 , with 1 value :
Size of Vector – 2
Value contained in vector – 1.0
Position of 1.0 value in vector – 0th place

 **Repeat the same procedure for the other categorical column
(Country)**

In [ ]:
country_indexer = StringIndexer(inputCol='Country',outputCol='Country_num').fit(df)
df = country_indexer.transform(df)


In [ ]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [ ]:
df.groupBy('Country_num').count().show()

+-----------+-----+
|Country_num|count|
+-----------+-----+
|        0.0|12178|
|        1.0| 4018|
|        3.0| 1218|
|        2.0| 2586|
+-----------+-----+



In [ ]:
country_vector =  OneHotEncoder(inputCol='Country_num',outputCol="Country_Vector")
df= country_vector.transform(df)

In [ ]:
df.select(['Country','Country_num','Country_Vector']).show(5)

+---------+-----------+--------------+
|  Country|Country_num|Country_Vector|
+---------+-----------+--------------+
|    India|        1.0| (3,[1],[1.0])|
|   Brazil|        2.0| (3,[2],[1.0])|
|   Brazil|        2.0| (3,[2],[1.0])|
|Indonesia|        0.0| (3,[0],[1.0])|
| Malaysia|        3.0|     (3,[],[])|
+---------+-----------+--------------+
only showing top 5 rows



In [ ]:
df.groupBy('Country_Vector').count().orderBy('count',ascending=False).show(5)

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
| (3,[0],[1.0])|12178|
| (3,[1],[1.0])| 4018|
| (3,[2],[1.0])| 2586|
|     (3,[],[])| 1218|
+--------------+-----+



Now that we have converted both the categorical columns into
numerical forms, we need to assemble all of the input columns into a
single vector that would act as the input feature for the model.

In [ ]:
df_assembler = VectorAssembler(inputCols=['Platform_vector','Country_Vector','Age','Repeat_Visitor','Web_pages_viewed'],outputCol='features')


In [ ]:
df = df_assembler.transform(df)


In [ ]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Platform_num: double (nullable = false)
 |-- Platform_vector: vector (nullable = true)
 |-- Country_num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



As we can see, now we have one extra column named features, which
is nothing but a combination of all the input features represented as a
single dense vector.

In [ ]:
df.select(['features','Status']).show(8)

+--------------------+------+
|            features|Status|
+--------------------+------+
|[1.0,0.0,0.0,1.0,...|     1|
|[1.0,0.0,0.0,0.0,...|     0|
|(8,[1,4,5,7],[1.0...|     0|
|(8,[2,5,6,7],[1.0...|     1|
|(8,[1,5,7],[1.0,3...|     1|
|(8,[1,4,5,7],[1.0...|     0|
|(8,[1,4,5,7],[1.0...|     0|
|(8,[1,2,5,7],[1.0...|     0|
+--------------------+------+
only showing top 8 rows



In [ ]:
model_df = df.select(['features','Status'])

## Step 5: Splitting the Dataset

In [ ]:
training_df,test_df = model_df.randomSplit([0.75,0.25])


In [ ]:
print(training_df.count())

15023


In [ ]:
training_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7581|
|     0| 7442|
+------+-----+



This ensures we have a balance set of the target class (Status) into the
training and test set.

In [ ]:
test_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2419|
|     0| 2558|
+------+-----+



Step **6**: Build and Train Logistic Regression Model

In this part, we build and train the logistic regression model using features
as the input column and status as the output column.

In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(labelCol='Status').fit(training_df)

## Training Results

We can access the predictions made by the model using the evaluate
function in Spark that executes all the steps in an optimized way. That
gives another Dataframe that contains four columns in total, including
prediction and probability. The prediction column signifies the class
label that the model has predicted for the given row and probability
column contains two probabilities (probability for negative class at 0th
index and probability for positive class at 1st index)

In [ ]:
train_results=log_reg.evaluate(training_df).predictions

In [ ]:
train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).select(['Status',
'prediction','probability']).show(5,False)

+------+----------+---------------------------------------+
|Status|prediction|probability                            |
+------+----------+---------------------------------------+
|1     |1.0       |[0.2988542166633331,0.7011457833366669]|
|1     |1.0       |[0.2988542166633331,0.7011457833366669]|
|1     |1.0       |[0.1664464152266984,0.8335535847733015]|
|1     |1.0       |[0.08554472838577191,0.914455271614228]|
|1     |1.0       |[0.08554472838577191,0.914455271614228]|
+------+----------+---------------------------------------+
only showing top 5 rows



So, in the above results, probability at the 0th index is for Status = 0
and probability as 1st index is for Status =1

### Step : Evaluate Linear Regression Model on Test Data

In [ ]:
results = log_reg.evaluate(test_df).predictions

In [ ]:
results.select(['Status','prediction']).show(20,False)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|1     |0.0       |
|0     |1.0       |
|1     |1.0       |
|0     |1.0       |
|1     |1.0       |
|1     |1.0       |
|1     |1.0       |
|0     |1.0       |
+------+----------+
only showing top 20 rows



Since this is a classification problem, we will use a confusion matrix to
gauge the performance of the model.

## Confusion Matrix

We will manually create the variables for true positives, true negatives,
false positives, and false negatives to understand them better rather than
using the direct inbuilt function.

In [ ]:
tp = results[(results.Status == 1) & (results.prediction== 1)].count()
tn = results[(results.Status == 0) & (results.prediction== 0)].count()
fp = results[(results.Status == 0) & (results.prediction== 1)].count()
fn = results[(results.Status == 1) & (results.prediction== 0)].count()

### Accuracy

In [ ]:
accuracy=float((tp+tn) /(results.count()))
print(accuracy)

0.9391199517781796


## Recall

In [ ]:
recall = float(tp)/(tp + fn)
print(recall)

0.9388176932616784


## Precision

In [ ]:
precision = float(tp) / (tp +fp)
print(precision)

0.9361088211046991
